In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('shopping_trends_updated.csv')

## 数据预览

In [ ]:
# 维度
data.shape

In [ ]:
# 数据信息
data.info()

In [ ]:
# 缺失值
data.isnull().sum()

In [ ]:
# 重复值
data.duplicated().sum()

In [ ]:
# 判断使用折扣和使用优惠码的情况是否同时出现
(data['Discount Applied'] == data['Promo Code Used']).all()

这里需要特别说明，当我们两列值完全相同则表明使用折扣就代表了使用优惠码

## 数据处理

In [ ]:
data.head()

In [ ]:
## 修改CustomerID格式
data['Customer ID'] = data['Customer ID'].astype(str)

In [ ]:
# 删除使用优惠码的那一列（原因：使用优惠码的情况已经在上一步判断过了）
data.drop(['Promo Code Used'], axis = 1, inplace = True)

## 数据探索

### 客户信息分布

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize = (15, 7))
plt.subplot(1, 2, 1)
sns.distplot(data['Age'], kde = False, bins = 20)
plt.title(f'Age Distribution', fontsize = 15)
plt.subplot(1, 2, 2)
sns.countplot(data = data, x = 'Gender', order = data['Gender'].value_counts().index)
plt.title(f'Gender Distribution', fontsize = 15)
plt.xlabel('Gender', fontsize = 12)
plt.ylabel('Count', fontsize = 12)
plt.xticks(rotation = 0)
plt.tight_layout()
plt.show()

得到信息：男性客户约为女性客户的两倍，客户年龄分布较为均匀，主要是18 - 70岁

### 购物行为分析

In [ ]:
# 1. 商品类别的销售情况
category_sales = data.groupby('Category')['Purchase Amount (USD)'].sum().sort_values(ascending = False)
# 2. 商品类别的订单量
category_popularity = data['Category'].value_counts()
# 3. 不同类别商品的平均消费金额
category_avg_purchase = data.groupby('Category')['Purchase Amount (USD)'].mean().sort_values(ascending = False)
# 4. 客户最常用的支付方式
payment_method_counts = data['Payment Method'].value_counts()

fig, axes = plt.subplots(2, 2, figsize = (14, 10))
fig.tight_layout(pad = 5.0)
# 商品类别销售总额条形图
sns.barplot(x = category_sales.index, y = category_sales.values, ax = axes[0, 0])
axes[0, 0].set_title('Total Sales by Category')
axes[0, 0].set_xlabel('Category')
axes[0, 0].set_ylabel('Sales (USD)')
# 商品订单情况条形图
sns.barplot(x = category_popularity.index, y = category_popularity.values, ax = axes[0, 1])
axes[0, 1].set_title('Popularity by Category')
axes[0, 1].set_xlabel('Category')
axes[0, 1].set_ylabel('Number of Orders')
# 绘制商品类别的平均消费金额的条形图
sns.barplot(x=category_avg_purchase.index, y=category_avg_purchase.values, ax=axes[1, 0])
axes[1, 0].set_title('Average Purchase Amount by Category')
axes[1, 0].set_xlabel('Category')
axes[1, 0].set_ylabel('Average Purchase Amount (USD)')

# 绘制客户最常用的支付方式的条形图
sns.barplot(x=payment_method_counts.index, y=payment_method_counts.values, ax=axes[1, 1])
axes[1, 1].set_title('Usage of Payment Methods')
axes[1, 1].set_xlabel('Payment Method')
axes[1, 1].set_ylabel('Number of Orders')
plt.show()

In [ ]:
# 1.不同地点的销售总额
location_sales = data.groupby('Location')['Purchase Amount (USD)'].sum().sort_values(ascending=False)

# 2.不同地点的订单数量
location_order_count = data['Location'].value_counts()

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 16))
fig.tight_layout(pad=5.0)

# 绘制不同地点的销售总额条形图
sns.barplot(y=location_sales.index, x=location_sales.values, ax=axes[0])
axes[0].set_title('Total Sales by Location')
axes[0].set_xlabel('Total Sales (USD)')
axes[0].set_ylabel('Location')

# 绘制不同地点的订单数量条形图
sns.barplot(y=location_order_count.index, x=location_order_count.values, ax=axes[1])
axes[1].set_title('Number of Orders by Location')
axes[1].set_xlabel('Number of Orders')
axes[1].set_ylabel('Location')
plt.show()

In [ ]:
# 1.不同季节的销售总额
season_sales = data.groupby('Season')['Purchase Amount (USD)'].sum().sort_values(ascending=False)

# 2.不同季节的订单数量
season_order_count = data['Season'].value_counts()

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 8))
fig.tight_layout(pad=5.0)

# 绘制不同季节的销售总额条形图
sns.barplot(x=season_sales.index, y=season_sales.values, ax=axes[0])
axes[0].set_title('Total Sales by Season')
axes[0].set_xlabel('Season')
axes[0].set_ylabel('Total Sales (USD)')

# 绘制不同季节的订单数量条形图
sns.barplot(x=season_order_count.index, y=season_order_count.values, ax=axes[1])
axes[1].set_title('Number of Orders by Season')
axes[1].set_xlabel('Season')
axes[1].set_ylabel('Number of Orders')
plt.show()

In [ ]:
# 1.有折扣和没有折扣的订单的平均消费金额
avg_purchase_discount = data.groupby('Discount Applied')['Purchase Amount (USD)'].mean()

# 2.有折扣和没有折扣的订单数量
purchase_counts = data.groupby('Discount Applied').size()

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 8))
fig.tight_layout(pad=5.0)

# 绘制有折扣和没有折扣的订单的平均消费金额条形图
sns.barplot(x=avg_purchase_discount.index, y=avg_purchase_discount.values, ax=axes[0])
axes[0].set_title('Average Purchase Amount with/without Discount')
axes[0].set_xlabel('Discount Applied')
axes[0].set_ylabel('Average Purchase Amount (USD)')

# 绘制有折扣和没有折扣的订单数量条形图
sns.barplot(x=purchase_counts.index, y=purchase_counts.values, ax=axes[1])
axes[1].set_title('Number of Orders with/without Discount')
axes[1].set_xlabel('Discount Applied')
axes[1].set_ylabel('Number of Orders')
plt.show()

通过我们建立的可视化可以看出以下几点：
1. 衣服的销售总量和订单都是最高的，外套的销售总量和订单都是最低的，说明外套作为不常更换商品，销量无法比拟其他。
2. 鞋类、衣服、配饰三类商品的平均金额都比较接近，在$60左右，外套的平均消费金额最低，也符合我们提到的更新频率较低。
3. Paypal是最常用的支付方式，其次是信用卡和现金，最少的支付方式是银行转账。
4. 蒙大拿州（Montana）的销售总额和订单数量是最高的，堪萨斯州（Kansas）的销售总额和订单数量是最低的。
5. 秋季（Fall）有着最高的销售金额，春季（Spring）有着最高的订单数量，夏季（Summer）的销售金额和订单数量都是最低的。
6. 使用折扣的订单平均消费金额略小于未使用折扣的订单平均销售金额，未使用折扣的订单数量大于使用折扣的订单数量。

### 商品销售情况分析

In [ ]:
# 筛选出四个类别下的商品数据
categories_of_interest = ['Clothing', 'Accessories', 'Footwear', 'Outerwear']
filtered_data = data[data['Category'].isin(categories_of_interest)]

# 商品种类分析
item_counts = filtered_data.groupby('Category')['Item Purchased'].value_counts().unstack(fill_value=0)

# 创建堆叠条形图
ax = item_counts.plot(kind='bar', stacked=True, figsize=(16, 12), colormap='viridis', legend=False)

# 设置图表标题和坐标轴标签
ax.set_title('Sales Quantity of Different Items in Each Category', fontsize=16)
ax.set_xlabel('Category', fontsize=12)
ax.set_ylabel('Quantity Sold', fontsize=12)

# 优化坐标轴刻度显示
ax.yaxis.get_major_formatter().set_scientific(False)

# 在每个颜色段上标注商品名称和数量
for i, category in enumerate(item_counts.index):
    y_offset = 0
    for item, count in item_counts.loc[category].items():
        if count > 0:
            ax.text(i, y_offset + count / 2, f'{item}:{count}', ha='center', va='center', fontsize=12, color='white',fontweight='bold')
            y_offset += count

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# 计算每个商品在每个类别下的销售总额
sales_amount = filtered_data.groupby('Category').apply(lambda x: x.groupby('Item Purchased')['Purchase Amount (USD)'].sum())

# 将sales_amount转换为DataFrame并进行适当的调整
sales_amount_df = sales_amount.unstack(fill_value=0)

# 创建堆叠条形图
ax = sales_amount_df.plot(kind='bar', stacked=True, figsize=(16, 12), colormap='viridis', legend=False)

# 设置图表标题和坐标轴标签
ax.set_title('Sales Amount of Different Items in Each Category', fontsize=16)
ax.set_xlabel('Category', fontsize=12)
ax.set_ylabel('Sales Amount (USD)', fontsize=12)

# 优化坐标轴刻度显示
ax.yaxis.get_major_formatter().set_scientific(False)

# 在每个颜色段上标注商品名称和销售总额
for i, category in enumerate(sales_amount_df.index):
    y_offset = 0
    for item, amount in sales_amount_df.loc[category].items():
        if amount > 0:
            ax.text(i, y_offset + amount / 2, f'{item}:{amount}', ha='center', va='center', fontsize=12, color='white',fontweight='bold')
            y_offset += amount

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# 计算每个商品在每个类别下的平均销售金额
average_sales_amount = sales_amount_df.div(item_counts)

# 创建堆叠条形图
ax = average_sales_amount.plot(kind='bar', stacked=True, figsize=(16,12), colormap='viridis', legend=False)

# 设置图表标题和坐标轴标签
ax.set_title('Average Sales Amount of Different Items in Each Category', fontsize=16)
ax.set_xlabel('Category', fontsize=12)
ax.set_ylabel('Average Sales Amount (USD)', fontsize=12)

# 优化坐标轴刻度显示
ax.yaxis.get_major_formatter().set_scientific(False)

# 在每个颜色段上标注商品名称和平均销售金额
for i, category in enumerate(average_sales_amount.index):
    y_offset = 0
    for item, amount in average_sales_amount.loc[category].items():
        if not np.isnan(amount):
            ax.text(i, y_offset + amount / 2, f'{item}:{amount:.2f}', ha='center', va='center', fontsize=12, color='white', fontweight='bold')
            y_offset += amount

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

由此我们可以看出以下几点：
1. 在配饰（Accessories）中，订单数量最多且销售总额最高的商品是珠宝（Jewelry）。
2. 在衣服（Clothing）中，订单数量最多且销售总额最高的商品是（女式）短上衣（Blouse）。
3. 在鞋类（Footwear）中，订单数量最多的商品是凉鞋（Sandals），销售总额最高的商品是鞋（Shoes）。
4. 在外套（Outerwear）中，夹克（Jacket）和外套（Coat）二者相差不大。
5. 所有商品的平均消费金额都在56-63美元之间，相差不是很大，只是外套的平均消费金额均小于60美元。

## 客户忠诚度分析

### 基于购买频率和历史购买总数

In [ ]:
# 1.购买频率分析
frequency_counts = data['Frequency of Purchases'].value_counts(normalize=True) * 100

# 2.历史购买总数分析
historical_purchases_description = data['Previous Purchases'].describe()

# 3.联合分析：购买频率和历史购买总数
loyalty_analysis = data.groupby('Frequency of Purchases')['Previous Purchases'].agg(['count', 'mean', 'median', 'std'])

In [ ]:
# 购买频率分布条形图
plt.figure(figsize=(16, 12))
ax = sns.countplot(data=data, x='Frequency of Purchases', order = data['Frequency of Purchases'].value_counts().index)
ax.set_title('Customer Counts by Purchase Frequency')
ax.set_ylabel('Number of Customers')
ax.set_xlabel('Frequency of Purchases')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
plt.show()

In [ ]:
# 历史购买总数分布直方图
plt.figure(figsize=(16, 12))
ax = sns.distplot(data['Previous Purchases'], bins=50, kde=False)
ax.set_title('Distribution of Previous Purchases')
ax.set_ylabel('Frequency')
ax.set_xlabel('Previous Purchases')
plt.show()

In [ ]:
# 联合分析：购买频率与历史购买总数的箱线图
plt.figure(figsize=(16, 12))
ax = sns.boxplot(data=data, x='Frequency of Purchases', y='Previous Purchases', order = data['Frequency of Purchases'].value_counts().index)
ax.set_title('Previous Purchases by Purchase Frequency')
ax.set_ylabel('Previous Purchases')
ax.set_xlabel('Frequency of Purchases')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
plt.show()

基于图表我们可以得出以下几点：
1.客户的购买频率分布相对均衡，没有明显的主导趋势。
2.大多数客户的历史购买总数集中在25附近，但也存在一些购买数量远高或远低于平均水平的客户。
3.高频率购买者（如每周或每两周购物）和高历史购买总数的客户可能是忠诚度较高的客户群体

### 基于评分和订阅状态

In [ ]:
# 1. 评分分布直方图
plt.figure(figsize=(16, 12))
ax = sns.distplot(data['Review Rating'], bins=15, kde=True)
ax.set_title('Distribution of Review Ratings')
ax.set_ylabel('Frequency')
ax.set_xlabel('Review Rating')
plt.show()

In [ ]:
# 2. 订阅状态与评分的关系的箱线图
plt.figure(figsize=(12, 9))
ax = sns.boxplot(data=data, x='Subscription Status', y='Review Rating')
ax.set_title('Review Ratings by Subscription Status')
ax.set_ylabel('Review Rating')
ax.set_xlabel('Subscription Status')

# 显示图表
plt.show()

基于图表我们可以得出以下几点：
1.评分集中在2.5到5之间，表明大多数客户对购买的商品感到满意，没有出现1分这种极端情况。
2.拥有订阅的客户和没有订阅的客户在评分分布上没有明显差异。

### 高评分客户的特征

这里将仅对评分大于等于4的客户作为高评分用户，对高评分客户进行分析

In [ ]:
high_rating_customers = data[data['Review Rating'] > 4]
# 1.购买频率分析
frequency_distribution_high_rating = high_rating_customers['Frequency of Purchases'].value_counts(normalize=True) * 100
frequency_distribution_high_rating

In [ ]:
# 2.购买金额分析
purchase_amount_description_high_rating = high_rating_customers['Purchase Amount (USD)'].describe()
purchase_amount_description_high_rating

In [ ]:
# 购买金额分布直方图
plt.figure(figsize=(16, 12))
ax = sns.distplot(high_rating_customers['Purchase Amount (USD)'], bins=30, kde=True)
ax.set_title('Distribution of Purchase Amount for High Rating Customers')
ax.set_ylabel('Frequency')
ax.set_xlabel('Purchase Amount (USD)')
plt.show()

In [ ]:
# 3.购买类别分析
category_distribution_high_rating = high_rating_customers['Category'].value_counts(normalize=True) * 100
category_distribution_high_rating

In [ ]:
# 4.订阅状态的客户分布
subscription_status_high_rating = high_rating_customers['Subscription Status'].value_counts(normalize=True) * 100
subscription_status_high_rating

In [ ]:
# 5.购买地点分析
location_distribution_high_rating = high_rating_customers['Location'].value_counts(normalize=True) * 100
location_distribution_high_rating.head(10)

由此我们可以得出以下几点：
1. 给出高评分的客户在购买频率和购买金额上没有明显的特殊趋势。
2. 给出高评分的客户主要购买服装和配饰类别的商品。
3. 大多数给出高评分的客户没有订阅服务。
4. 给出高评分的客户分布在各个地区，没有明显集中在特定地区。

## 基于矩阵分解模型建立商品推荐系统

为了实现矩阵分解模型，我们需要使用到奇异值分解法（SVD）来进行特征提取。

### 数据预处理

In [ ]:
# 将“购买频率”转换为数值
frequency_mapping = {
    'Weekly': 52,
    'Bi-Weekly': 26,
    'Monthly': 12,
    'Quarterly': 4,
    'Every 3 Months': 4,
    'Annually': 1,
    'Fortnightly': 26
}
data_copy = data.copy()
data_copy['Frequency of Purchases'] = data_copy['Frequency of Purchases'].map(frequency_mapping)
# 归一化“历史购买总数”和“购买频率”
scaler = MinMaxScaler()
data_copy[['Previous Purchases', 'Frequency of Purchases']] = scaler.fit_transform(data_copy[['Previous Purchases', 'Frequency of Purchases']])
data_copy.head()

### 创建评分矩阵，进行SVD分解

In [ ]:
from scipy.sparse import csr_matrix

# 创建评分矩阵
rating_matrix = data_copy.pivot(index='Customer ID', columns='Item Purchased', values='Review Rating').fillna(0)

# 将 DataFrame 转换为稀疏矩阵
rating_matrix_sparse = csr_matrix(rating_matrix.values)

# 进行SVD分解
# 归一化评分矩阵：减去每个用户的平均评分
user_ratings_mean = np.mean(rating_matrix, axis=1)
rating_matrix_normalized = rating_matrix - user_ratings_mean.values.reshape(-1, 1)

# 进行SVD
U, sigma, Vt = svds(rating_matrix_sparse, k=10)

# 将sigma从向量转换为对角矩阵
sigma = np.diag(sigma)

# 预测评分
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.values.reshape(-1, 1)
predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=rating_matrix.columns, index=rating_matrix.index)
predicted_ratings_df.head()

### 创建用户画像

In [ ]:
# 创建用户画像，包括历史购买总数和购买频率
user_profile = data_copy.groupby('Customer ID')[['Previous Purchases', 'Frequency of Purchases']].mean()

# 结合用户的平均评分
user_profile['Average Rating'] = user_ratings_mean
user_profile.head()

### 基于用户画像进行推荐

In [ ]:
# 结合用户画像
user_factors = np.dot(U, sigma)  # 用户矩阵
user_factors_df = pd.DataFrame(user_factors, index=rating_matrix.index)
user_factors_combined = user_factors_df.join(user_profile)

# 调整评分矩阵
adjusted_user_factors = user_factors_combined.apply(
    lambda x: x * (1 + x['Previous Purchases']) * (1 + x['Frequency of Purchases']) * x['Average Rating'], axis=1
)
adjusted_user_factors = adjusted_user_factors.drop(['Previous Purchases', 'Frequency of Purchases', 'Average Rating'], axis=1)

# 预测调整后的评分
adjusted_predicted_ratings = np.dot(adjusted_user_factors, Vt) + user_ratings_mean.values.reshape(-1, 1)
adjusted_predicted_ratings_df = pd.DataFrame(adjusted_predicted_ratings, columns=rating_matrix.columns, index=rating_matrix.index)

In [ ]:
def Recommendation_System(user_id,num_recommendations=5):
    adjusted_recommended_items = adjusted_predicted_ratings_df.loc[user_id].sort_values(ascending=False).head(num_recommendations)
    print(adjusted_recommended_items)

In [ ]:
Recommendation_System('1')
print('-'*50)
Recommendation_System('15')
print('-'*50)
Recommendation_System('20')
print('-'*50)
Recommendation_System('25')

通过推荐系统，这里给'1'号客户推荐(女式)短上衣（Blouse），给'15'号客户推荐外套（Coat）、袜子（Socks）和男士衬衣（Shirt），给'20'号客户推荐裤子（Pants），给'25'号客户推荐夹克（Jacket）。

In [ ]:
import random

customer_id_range = range(1, 1001)

# 随机选择五个客户 ID
random_customers = random.sample(customer_id_range, 15)

# 对随机选择的客户进行推荐系统测试
for customer_id in random_customers:
    Recommendation_System(str(customer_id))
    print('-' * 50)

再次经过随机验证表明我们的推荐系统是可行的。